In [2]:
import pandas as pd
import os
import sys

sys.path.append("../..")
from common.utils import timestep_csv_names
from common.constants import DATAFOLDER

root_dir = os.path.join(DATAFOLDER.data_root_path, "one_day_data")

In [7]:
_timestep_csv_names = timestep_csv_names(delta=10)

dates, rain_amounts, start_times, end_times = [], [], [], []
for year in os.listdir(root_dir):
    for month in os.listdir(root_dir + f"/{year}"):
        for date in os.listdir(root_dir + f"/{year}/{month}"):
            minimum_data_length = 40
            start_time = None
            end_time = None
            for csv_filename in _timestep_csv_names:
                csv_path = os.path.join(
                        root_dir,
                        year, month, date, csv_filename
                    )
                parquet_path = csv_path.replace(".csv", ".parquet.gzip")
                parquet_df = pd.read_parquet(parquet_path, engine="pyarrow")

                if parquet_df["hour-rain"].max() >= 5:
                    max_rainfall = parquet_df["hour-rain"].max()
                    if start_time is None:
                        start_time = csv_filename.replace(".csv", "")

                if start_time is not None and parquet_df["hour-rain"].max() < 5:
                    end_time = csv_filename.replace(".csv", "")
                    dates.append(date)
                    rain_amounts.append(max_rainfall)
                    start_times.append(start_time)
                    end_times.append(end_time)
                    
                    start_time = None
                    end_time = None
                    max_rainfall = 0

                if len(parquet_df.index) < minimum_data_length:
                    minimum_data_length = len(parquet_df.index)

            if minimum_data_length > 0 and start_time is not None and end_time is None:
                dates.append(date)
                rain_amounts.append(max_rainfall)
                start_times.append(start_time)
                end_times.append(csv_filename.replace(".csv", ""))

df = pd.DataFrame({"date": dates, "rain": rain_amounts, "start_time": start_times, "end_time": end_times})
df = df.sort_values(by="rain", ascending=False)
df.reset_index()
df.head()

,date,rain,start_time,end_time
34,2020-09-01,20.5,23-30,23-50.csv
274,2020-05-29,19.5,13-10,14-20
160,2020-11-03,19.0,7-10,10-0
56,2020-09-14,19.0,4-50,7-0
73,2020-09-19,17.5,5-40,7-40


In [9]:
df.to_csv("train_dataset.csv")